# Scene Classification

## 3. Build Model-InceptionV3 TrainImageFromFolder-Top10Inc
- Import pkg
- Load sample data, only first 1000 objects
- 
Reference:
- https://challenger.ai/competitions
- https://github.com/jupyter/notebook/issues/2287

**Tensorboard**
1. Input at command:  **tensorboard --logdir=./log**
2. Input at browser:  **http://127.0.0.1:6006**

### Import pkg

In [1]:
import numpy as np
import pandas as pd
# import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
# import zipfile
import os
import zipfile
import math
import time
from IPython.display import display
import pdb
import json
from PIL import Image
import glob
import pickle

### Load sample data, only first 1000 objects

In [ ]:
input_path = './input'
datasetName = 'train'
date = '20170904'

zip_path = input_path + '/ai_challenger_scene_{0}_{1}.zip'.format(datasetName, date)
extract_path = input_path + '/ai_challenger_scene_{0}_{1}'.format(datasetName, date)
image_path = extract_path + '/scene_{0}_images_{1}'.format(datasetName, date)
scene_classes_path = extract_path + '/scene_classes.csv'
scene_annotations_path = extract_path + '/scene_{0}_annotations_{1}.json'.format(datasetName, date)

print(input_path)
print(zip_path)
print(extract_path)
print(image_path)
print(scene_classes_path)
print(scene_annotations_path)

In [ ]:
scene_classes = pd.read_csv(scene_classes_path, header=None)
display(scene_classes.head())

In [ ]:
def get_scene_name(lable_number, scene_classes_path):
    scene_classes = pd.read_csv(scene_classes_path, header=None)
    return scene_classes.loc[lable_number, 2]
print(get_scene_name(0, scene_classes_path))

In [ ]:
def load_pickle_data(dataset, index):
    pickleFolder = './input/pickle_{0}'.format(dataset)
    print(pickleFolder)
    x_path = pickleFolder + '/x_data{0}.p'.format(index)
    y_path = pickleFolder + '/y_data{0}.p'.format(index)
    print(x_path)
    print(y_path)
    if not os.path.exists(x_path):
        print(x_path + ' do not exist!')
        return
    if not os.path.exists(y_path):
        print(y_path + ' do not exist!')
        return
    x_data = pickle.load(open(x_path, mode='rb'))
    y_data = pickle.load(open(y_path, mode='rb'))
#     y_data = to_categorical(y_train)
    print(x_data.shape)
    print(y_data.shape)
    return (x_data, y_data)

x_train, y_train = load_pickle_data("train", 0)
print(x_train.shape)
print(y_train.shape)
del x_train
del y_train

In [ ]:
%%time
x_train, y_train = load_pickle_data("train", 0)
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(x_train[0])
ax[0].set_title(get_scene_name(y_train[0], scene_classes_path))
ax[1].imshow(x_train[1])
ax[1].set_title(get_scene_name(y_train[1], scene_classes_path))
del x_train
del y_train

In [ ]:
%%time
x_val, y_val = load_pickle_data("validation", 0)
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(x_val[0])
ax[0].set_title(get_scene_name(y_val[0], scene_classes_path))
ax[1].imshow(x_val[1])
ax[1].set_title(get_scene_name(y_val[1], scene_classes_path))
del x_val
del y_val

### Load model

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3

In [ ]:
%%time
model = load_model('./model/TrainImageFolder2017-10-02_13-35-13/TrainImageFolder2017-10-02_13-35-13.h5')
x_val, y_val = load_pickle_data("validation", 0)
y_val = to_categorical(y_val)
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))
del x_val, y_val

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers[:41]:
    layer.trainable = False
for layer in model.layers[41:]:
    layer.trainable = True

model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [ ]:
def saveModel(model, middleName):
    modelPath = './model'
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
        
    fileName = middleName + time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
    jsonFileName = modelPath + '/' + fileName + '.json'
    yamlFileName = modelPath + '/' + fileName + '.yaml'

    json_string = model.to_json()
    with open(jsonFileName, 'w') as file:
        file.write(json_string)
    yaml_string = model.to_yaml()
    with open(yamlFileName, 'w') as file:
        file.write(yaml_string)
    
    weigthsFile = modelPath + '/' + fileName + '.h5'
    model.save(weigthsFile)
# saveModel(model, 'ModelSaveTest')

**Train top 2 inception**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 20,
                                   zoom_range = 0.1,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   vertical_flip = True)
train_generator = train_datagen.flow_from_directory('./input/data_train',
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode = "categorical")
print(train_generator.classes[0:1000])

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
tensorBoard = TensorBoard(log_dir='./log_Top2Inc_171002-3')

In [ ]:
%%time
x_val, y_val = load_pickle_data("validation", 0)
y_val = to_categorical(y_val)

# model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])
hist = model.fit_generator(train_generator,
                       steps_per_epoch=128,
                       epochs=32, #Increase this when not on Kaggle kernel
                       verbose=2,  #1 for ETA, 0 for silent
                       validation_data=(x_val, y_val),
                       callbacks=[annealer, tensorBoard])

In [ ]:
saveModel(model, 'TrainImageFolder')

In [ ]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
print('Done!')